In [ ]:
import os
from pathlib import Path

import pandas as pd
import tomli
from dotenv import load_dotenv
from openai import AzureOpenAI

from discharge_docs.dashboard.helper import (
    get_data_from_patient_admission,
    get_patients_from_list_names,
    get_template_prompt,
)
from discharge_docs.processing.processing import (
    get_patient_file,
)
from discharge_docs.prompts.prompt import (
    load_prompts,
    load_template_prompt,
)
from discharge_docs.prompts.prompt_builder import PromptBuilder

%load_ext autoreload
%autoreload 2

# initialise Azure
load_dotenv()
TEMPERATURE = 0.2

# deployment_name = "aiva-gpt" # GPT 3.5
deployment_name = "aiva-gpt4"
client = AzureOpenAI(
    api_version="2024-02-01",
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT", ""),
)
# load data
df_metavision = pd.read_parquet(
    Path.cwd().parent / "data" / "processed" / "metavision_new_data.parquet"
)

df_HIX = pd.read_parquet(Path.cwd().parent / "data" / "processed" / "HiX_data.parquet")

# Define your DataFrames for each department
df_dict = {
    "NICU": df_metavision,
    "IC": df_metavision,
    "CAR": df_HIX,
    "PSY": df_HIX,
}


# load used enc_ids
with open(
    Path.cwd().parent
    / "src"
    / "discharge_docs"
    / "dashboard"
    / "enc_ids_dashboard.toml",
    "rb",
) as f:
    enc_ids_dict = tomli.load(f)
    for key in enc_ids_dict:
        enc_ids_dict[key] = enc_ids_dict[key]["ids"]

for key in enc_ids_dict:
    enc_ids_dict[key] = enc_ids_dict[key][:25]
print(enc_ids_dict)

data_dict, values_list = get_patients_from_list_names(df_dict, enc_ids_dict)

# load prompts
user_prompt, system_prompt = load_prompts()
template_prompt_NICU = load_template_prompt("NICU")
template_prompt_IC = load_template_prompt("IC")
template_prompt_CAR = load_template_prompt("CAR")
template_prompt_PSY = load_template_prompt("PSY")
template_prompt_dict = {
    "nicu": template_prompt_NICU,
    "ic": template_prompt_IC,
    "car": template_prompt_CAR,
    "psy": template_prompt_PSY,
    "demo": template_prompt_NICU,
}

# selected_patient_admission = list(data_dict.keys())[0]
selected_patient_admission = 'patient_15_ic'

print(selected_patient_admission)
patient_data = get_data_from_patient_admission(
    selected_patient_admission, data_dict
)

template_prompt, department = get_template_prompt(
    selected_patient_admission, template_prompt_dict
)

prompt_builder = PromptBuilder(
    temperature=TEMPERATURE, deployment_name=deployment_name, client=client
)

In [ ]:
template_prompt = """Er is een categorie: 'Beloop'. 

Schrijf onder 'Beloop' verhalenderwijs een chronologisch beloop van de opname met complicaties, beleid en beweegredenen voor het beleid. Wees duidelijk in welke beweegredenen belangrijk waren. Schrijf beknopt en focus op diagnoses in plaatst van symptomen.
Het is belangrijk dat zowel wat over de respiratie als de hemodynamiek gedocumenteerd wordt. Schrijf het beloop in maximaal 300 woorden op."""

In [ ]:
patient_file_string, _ = get_patient_file(patient_data)
discharge_letter = prompt_builder.generate_discharge_doc(
    patient_file=patient_file_string,
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    template_prompt=template_prompt,
)
generated_doc = discharge_letter
print(generated_doc)

In [ ]:
for category_pair in generated_doc:
    print(category_pair["Categorie"])
    print(category_pair["Beloop tijdens opname"])